## Data Ingestion stage

This notebook handles the **data ingestion stage** of the project. Its main purpose is to:
- Download the raw dataset (cats and dogs images) from a specified URL.
- Unzip and extract the images.
- Clean the dataset by removing corrupted or empty files.
- Prepare the data directory for further processing and model training.

In [1]:
# Import required modules for data handling
from collections import namedtuple
import os

In [2]:
# (Optional) Change working directory up one level if needed
os.chdir("../")

In [3]:
# Print the current working directory to confirm location
%pwd

'/home/zkhechadoorian/CNNs_Cats_and_Dogs'

In [4]:
# Define a namedtuple for data ingestion configuration
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

# Define a dataclass for data ingestion configuration (alternative to namedtuple)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
# Import utility functions and constants for configuration management
from ImageClassification.constants import *
from ImageClassification.utils import read_yaml, create_directories

The `ConfigurationManager` class is responsible for managing the configuration of the data ingestion pipeline. It reads configuration files, parses their contents, and provides structured access to the configuration data. Here's a detailed explanation of the `ConfigurationManager` class and its relevant variables:

### **ConfigurationManager Class**
The `ConfigurationManager` class is defined in **CELL INDEX: 9**. Its primary purpose is to:
1. **Read Configuration Files**: It reads the configuration file (`config/config.yaml`) and parameters file (`params.yaml`) using the `read_yaml` utility function.
2. **Create Necessary Directories**: It ensures that the directories specified in the configuration file are created using the `create_directories` utility function.
3. **Provide Configuration Objects**: It provides structured access to the configuration data by returning instances of the `DataIngestionConfig` class.

#### **Constructor (`__init__`)**
The constructor initializes the `ConfigurationManager` object with the following:
- `config_filepath`: Path to the configuration file (default: `CONFIG_FILE_PATH`).
- `params_filepath`: Path to the parameters file (default: `PARAMS_FILE_PATH`).

It performs the following actions:
1. Reads the configuration and parameters files using `read_yaml`.
2. Creates the root directory for artifacts as specified in the configuration file.

#### **Method: `get_data_ingestion_config`**
This method:
1. Extracts the `data_ingestion` section from the configuration file.
2. Creates the root directory for data ingestion artifacts.
3. Returns an instance of the `DataIngestionConfig` class, which contains the following attributes:
    - `root_dir`: Root directory for data ingestion artifacts.
    - `source_URL`: URL of the dataset to be downloaded.
    - `local_data_file`: Path to the local file where the dataset will be saved.
    - `unzip_dir`: Directory where the dataset will be extracted.

---

### **Relevant Variables**
The `ConfigurationManager` class relies on the following variables, which are defined in the notebook:

#### **1. CONFIG_FILE_PATH**
- **Type**: `pathlib.PosixPath`
- **Value**: `PosixPath('config/config.yaml')`
- **Purpose**: Specifies the path to the main configuration file (`config.yaml`), which contains settings for the data ingestion pipeline.

#### **2. PARAMS_FILE_PATH**
- **Type**: `pathlib.PosixPath`
- **Value**: `PosixPath('params.yaml')`
- **Purpose**: Specifies the path to the parameters file (`params.yaml`), which may contain additional settings or hyperparameters for the pipeline.

#### **3. config**
- **Type**: `ConfigurationManager`
- **Value**: An instance of the `ConfigurationManager` class.
- **Purpose**: Provides access to the configuration data and methods for managing the pipeline.

#### **4. create_directories**
- **Type**: Function
- **Purpose**: Utility function to create directories specified in the configuration file.

#### **5. read_yaml**
- **Type**: Function
- **Purpose**: Utility function to read YAML files and parse their contents into Python dictionaries.

---

### **Repository Structure**
The relevant variables (`CONFIG_FILE_PATH`, `PARAMS_FILE_PATH`) point to files stored in the repository. Here's where they are located:

1. **`config/config.yaml`**:
    - This file contains the main configuration for the pipeline, including paths, URLs, and other settings.
    - Example structure:
      ```yaml
      data_ingestion:
         root_dir: artifacts/data_ingestion
         source_URL: https://github.com/entbappy/Branching-tutorial/raw/master/cat-dog-data.zip
         local_data_file: artifacts/data_ingestion/data.zip
         unzip_dir: artifacts/data_ingestion
      artifacts_root: artifacts
      ```

2. **`params.yaml`**:
    - This file contains additional parameters or hyperparameters for the pipeline.
    - Example structure:
      ```yaml
      batch_size: 32
      learning_rate: 0.001
      epochs: 10
      ```

---

### **Relevance**
The `ConfigurationManager` class and its associated variables are critical for ensuring that the pipeline operates in a structured and reproducible manner. By centralizing configuration management, the pipeline becomes easier to maintain, debug, and extend.

In [6]:
# Define a configuration manager to read config files and create directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initialize the ConfigurationManager by reading configuration and parameter YAML files,
        and creating the root artifact directory.
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Extract the data_ingestion section from the configuration file,
        create the root directory for data ingestion artifacts,
        and return a DataIngestionConfig object.
        Returns:
            DataIngestionConfig: Configuration for data ingestion.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
# Import additional modules for downloading and extracting files
import os
import urllib.request as request
from zipfile import ZipFile

In [8]:
# Define the DataIngestion class for downloading and cleaning the dataset
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize the DataIngestion class with the given configuration.
        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config

    def download_file(self):
        """
        Download the dataset from the source URL if it does not already exist locally.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        """
        Filter the list of files to include only valid image files (cats and dogs).
        Args:
            list_of_files (list): List of file names from the zip archive.
        Returns:
            list: Filtered list of valid image file names.
        """
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        """
        Extract and clean an image file from the zip archive.
        Removes the file if it is empty or corrupted.
        Args:
            zf (ZipFile): ZipFile object for the dataset archive.
            f (str): File name to extract and preprocess.
            working_dir (str): Directory to extract the file to.
        """
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        """
        Unzip the dataset and clean corrupted or empty files.
        Only valid image files are extracted and cleaned.
        """
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [9]:
# Print the current working directory again (for debugging)
%pwd

'/home/zkhechadoorian/CNNs_Cats_and_Dogs'

In [10]:
# Run the data ingestion pipeline: download and extract/clean the dataset
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2025-09-19 23:05:22,456: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-19 23:05:22,458: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-19 23:05:22,459: INFO: common: created directory at: artifacts]
[2025-09-19 23:05:22,459: INFO: common: created directory at: artifacts/data_ingestion]
